In [1]:
import igraph as g
from HLL import HyperLogLog
import numpy as np

In [2]:
def getLcc(filename, isDirected=True):
    data = g.Graph.Read_Ncol(filename, isDirected)
    if (isDirected):
        clust = data.clusters(mode='strong')
    else:
        clust = data.clusters(mode='weak')

    return clust.giant()


def getPaths(ig, isDirected=True):
    if (isDirected):
        return np.array(ig.shortest_paths())
    else:
        return np.array(ig.shortest_paths(mode='all'))

In [3]:
lcc = getLcc('data/wiki-Vote.txt')
print("nodes of lcc: ", lcc.vcount())
print("edges of lcc: ", lcc.ecount())

paths = getPaths(lcc)
print("median distance: ", np.median(paths))
print("mean distance: ", np.average(paths))
print("diameter: ", np.max(paths))
print("effective diameter: ", np.percentile(paths, 90))

nodes of lcc:  1300
edges of lcc:  39456
median distance:  3.0
mean distance:  2.87706804734
diameter:  9
effective diameter:  4.0


In [4]:
import random
for j in range(10):
    hll = HyperLogLog(16) # use 2^16 registers
    for i in range(100000):
        hll.add(str(random.random()))
    print(100000/hll.cardinality()) # ~4% err

0.9996558197724535
0.9963934480099572
1.0053656259373343
0.9937779799589113
0.9938238377584677
1.0009897874456124
0.9939155563073244
0.9984146072973453
1.003245657197983
1.0021864776668332


In [5]:
hll1 = HyperLogLog(16)
hll2 = HyperLogLog(16)
for i in range(100000):
    hll1.add(str(random.random()))
for i in range(100000):
    hll2.add(str(random.random()))
for i in range(100000):
    r = str(random.random())
    hll1.add(r)
    hll2.add(r)

print(hll1.cardinality())
print(hll2.cardinality())

hll1.merge(hll2)
print(hll1.cardinality())

201784.0456392771
201287.67758546027
300308.9220856516


In [6]:
def ANF(lcc, n_bucket_bits=5, mode='ALL'):
    """mode=IN/OUT/ALL"""
    chain = [0]
    vset = list()
    vset2 = list()
    for v in lcc.vs:
        hll = HyperLogLog(n_bucket_bits)
        hll.add(str(v.index))
        vset.append(hll)
        
        hll = HyperLogLog(n_bucket_bits)
        vset2.append(hll)

    for _iter in range(1, lcc.vcount()):
        for v in lcc.vs:
        #for v in tqdm(lcc.vs):
            vi = v.index
            vset2[vi].merge(vset[vi])
            for vn in v.neighbors(mode=mode): # mode=IN/OUT/ALL
                vni = vn.index
                vset2[vi].merge(vset[vni])
        vset, vset2 = vset2, vset

        k_deg_paths = sum([hll.cardinality() for hll in vset])
        k_deg_paths -= sum(chain)
        chain.append(k_deg_paths)

        # terminal point
        print('iteration: {}'.format(_iter))
        #if sum(chain) > lcc.vcount()**2 * 0.9:
        if chain[-1] < 1e-9:
            break
    
    return [i/2 for i in chain]

In [10]:
def percentile(chain, p):
    s = sum(chain)
    cur = 0
    for i in range(1, len(chain)):
        cur += chain[i]
        if cur / s >= p:
            return i

In [11]:
%time chain = ANF(lcc, n_bucket_bits=6, mode='OUT')
print(chain)
print("median distance: ", percentile(chain, 0.5))
print("mean distance: ", sum([i*n for i,n in enumerate(chain)]) / sum(chain))
print("diameter: ", len(chain)-1)
print("effective diameter: ", percentile(chain, 0.9))

iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
CPU times: user 679 ms, sys: 11.4 ms, total: 690 ms
Wall time: 796 ms
[0.0, 20391.572424914935, 228412.24650651717, 372984.9754629431, 145744.0276978946, 24918.929126088507, 2806.632742370828, 692.2283045685617, 388.5110832309583, 94.74496624909807, 0.0]
median distance:  3
mean distance:  2.9247725754380407
diameter:  10
effective diameter:  4
